# Goal:
Perform data carpentry on our basic boxscore dataset and insert into our database

Import libraries to support dataframe loading and manipulation as well as database design.

In [1]:
import pandas as pd
import getpass
import numpy as np
import time
from datetime import datetime
import psycopg2
from psycopg2.extensions import adapt, register_adapter, AsIs
pd.set_option('display.max_columns', 500)

Reading in datasets

In [2]:
data = pd.read_csv('../Data/nba_player_boxscores.csv')

In [3]:
boxscores = pd.read_csv('../Data/nba_boxscores.csv')

In [4]:
data = data.merge(boxscores[['boxscore','season']],on='boxscore')

In [7]:
data.head()

,assist_percentage,assists,block_percentage,blocks,box_plus_minus,boxscore,defensive_rating,defensive_rebound_percentage,defensive_rebounds,effective_field_goal_percentage,field_goal_attempts,field_goal_percentage,field_goals,free_throw_attempt_rate,free_throw_attempts,free_throw_percentage,free_throws,minutes_played,offensive_rating,offensive_rebound_percentage,offensive_rebounds,personal_fouls,player_id,points,steal_percentage,steals,team_abbr,three_point_attempt_rate,three_point_attempts,three_point_percentage,three_pointers,total_rebound_percentage,total_rebounds,true_shooting_percentage,turnover_percentage,turnovers,two_point_attempts,two_point_percentage,two_pointers,usage_percentage,venue,season
0,9.0,2.0,10.5,4.0,9.0,201810160BOS,80.0,13.1,4.0,0.571,7.0,0.571,4.0,0.143,1.0,1.0,1.0,29.950000,90.0,0.0,0.0,1.0,horfoal01,9.0,0.0,0.0,BOS,0.143,1.0,0.000,0.0,6.3,4.0,0.605,28.7,3.0,6.0,0.667,4.0,14.3,Home,2019
1,29.9,7.0,0.0,0.0,9.0,201810160BOS,89.0,13.5,4.0,0.179,14.0,0.143,2.0,0.143,2.0,1.0,2.0,29.000000,63.0,0.0,0.0,1.0,irvinky01,7.0,0.0,0.0,BOS,0.571,8.0,0.125,1.0,6.5,4.0,0.235,16.8,3.0,6.0,0.167,1.0,25.3,Home,2019
2,18.4,3.0,0.0,0.0,6.0,201810160BOS,83.0,23.7,7.0,0.559,17.0,0.529,9.0,0.294,5.0,0.8,4.0,28.933333,125.0,6.3,2.0,2.0,tatumja01,23.0,1.6,1.0,BOS,0.294,5.0,0.200,1.0,14.6,9.0,0.599,5.0,1.0,12.0,0.667,8.0,28.6,Home,2019
3,10.1,2.0,0.0,0.0,11.0,201810160BOS,89.0,6.9,2.0,0.423,13.0,0.385,5.0,0.154,2.0,0.5,1.0,28.383333,91.0,9.6,3.0,4.0,brownja02,12.0,0.0,0.0,BOS,0.231,3.0,0.333,1.0,8.3,5.0,0.432,12.6,2.0,10.0,0.400,4.0,22.9,Home,2019
4,0.0,0.0,0.0,0.0,10.0,201810160BOS,74.0,11.9,3.0,0.375,12.0,0.333,4.0,0.167,2.0,0.5,1.0,24.616667,90.0,7.4,2.0,1.0,haywago01,10.0,7.5,4.0,BOS,0.250,3.0,0.333,1.0,9.6,5.0,0.388,0.0,0.0,9.0,0.333,3.0,21.4,Home,2019


In [19]:
career = pd.read_csv('../Data/player_career_stats.csv')

In [20]:
career.head()

,season,and_ones,assist_percentage,assists,block_percentage,blocking_fouls,blocks,box_plus_minus,center_percentage,defensive_box_plus_minus,defensive_rebound_percentage,defensive_rebounds,defensive_win_shares,dunks,effective_field_goal_percentage,field_goal_attempts,field_goal_perc_sixteen_foot_plus_two_pointers,field_goal_perc_ten_to_sixteen_feet,field_goal_perc_three_to_ten_feet,field_goal_perc_zero_to_three_feet,field_goal_percentage,field_goals,free_throw_attempt_rate,free_throw_attempts,free_throw_percentage,free_throws,games_played,games_started,half_court_heaves,half_court_heaves_made,height,lost_ball_turnovers,minutes_played,nationality,net_plus_minus,offensive_box_plus_minus,offensive_fouls,offensive_rebound_percentage,offensive_rebounds,offensive_win_shares,on_court_plus_minus,other_turnovers,passing_turnovers,percentage_field_goals_as_dunks,percentage_of_three_pointers_from_corner,percentage_shots_three_pointers,percentage_shots_two_pointers,percentage_sixteen_foot_plus_two_pointers,percentage_ten_to_sixteen_footers,percentage_three_to_ten_footers,percentage_zero_to_three_footers,personal_fouls,player_efficiency_rating,player_id,point_guard_percentage,points,points_generated_by_assists,position,power_forward_percentage,salary,shooting_distance,shooting_fouls,shooting_fouls_drawn,shooting_guard_percentage,shots_blocked,small_forward_percentage,steal_percentage,steals,take_fouls,team_abbreviation,three_point_attempt_rate,three_point_attempts,three_point_percentage,three_point_shot_percentage_from_corner,three_pointers,three_pointers_assisted_percentage,total_rebound_percentage,total_rebounds,true_shooting_percentage,turnover_percentage,turnovers,two_point_attempts,two_point_percentage,two_pointers,two_pointers_assisted_percentage,usage_percentage,value_over_replacement_player,weight,win_shares,win_shares_per_48_minutes,player_name,player_season_number,birth_date,age
0,1976,NaN,0.104,4.0,0.000,NaN,0.0,-12.2,0.0,-6.2,0.114,7.0,0.0,NaN,NaN,46.0,NaN,NaN,NaN,NaN,0.370,17.0,0.065,3.0,1.000,3.0,13.0,NaN,NaN,NaN,74,NaN,64.0,United States of America,NaN,-6.0,NaN,0.016,1.0,-0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.0,7.3,mayfike01,0.0,37.0,NaN,SG,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.000,0.0,NaN,NYK,NaN,NaN,NaN,NaN,NaN,NaN,0.064,8.0,0.391,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.2,185,-0.2,-0.134,Ken Mayfield,1,1948-05-11,27.641909
1,2006,NaN,0.052,63.0,0.009,NaN,24.0,-3.7,0.0,-1.9,0.165,261.0,0.7,47.0,0.455,531.0,0.395,0.314,0.152,0.610,0.443,235.0,0.476,253.0,0.747,189.0,79.0,7.0,0.0,0.0,81,NaN,1952.0,United States of America,NaN,-1.7,NaN,0.073,122.0,2.1,NaN,NaN,NaN,0.100,0.208,0.100,0.900,0.386,0.066,0.062,0.386,227.0,12.2,willima02,0.0,672.0,NaN,PF,0.0,3883560.0,11.3,NaN,NaN,0.0,NaN,0.0,0.013,48.0,NaN,ATL,0.100,53.0,0.245,0.000,13.0,0.923,0.118,383.0,0.523,0.114,83.0,478.0,0.464,222.0,0.725,0.169,-0.8,237,2.8,0.069,Marvin Williams,1,1986-06-19,19.537704
2,2007,NaN,0.097,121.0,0.011,NaN,30.0,-2.7,0.0,-0.7,0.142,253.0,1.4,25.0,0.441,706.0,0.427,0.370,0.484,0.485,0.433,306.0,0.375,265.0,0.815,216.0,64.0,63.0,0.0,0.0,81,NaN,2179.0,United States of America,NaN,-2.0,NaN,0.046,84.0,0.9,NaN,NaN,NaN,0.044,0.222,0.064,0.936,0.477,0.076,0.091,0.292,191.0,12.2,willima02,0.0,839.0,NaN,PF,0.0,4174800.0,12.5,NaN,NaN,0.0,NaN,0.0,0.013,52.0,NaN,ATL,0.064,45.0,0.244,0.300,11.0,0.909,0.093,337.0,0.510,0.134,127.0,661.0,0.446,295.0,0.756,0.200,-0.4,237,2.3,0.051,Marvin Williams,2,1986-06-19,20.537040
3,2008,NaN,0.083,137.0,0.009,NaN,33.0,-1.6,0.0,-0.6,0.142,338.0,2.1,72.0,0.462,918.0,0.454,0.288,0.393,0.586,0.462,424.0,0.446,409.0,0.822,336.0,80.0,80.0,1.0,0.0,81,NaN,2765.0,United States of America,NaN,-1.0,NaN,0.050,119.0,3.6,NaN,NaN,NaN,0.087,0.500,0.011,0.989,0.475,0.113,0.117,0.284,225.0,14.5,willima02,0.0,1185.0,NaN,PF,0.0,4466040.0,11.6,NaN,NaN,0.0,NaN,0.0,0.015,81.0,NaN,ATL,0.011,10.0,0.100,0.000,1.0,1.000,0.096,457.0,0.540,0.104,127.0,908.0,0.466,423.0,0.754,0.201,0.3,237,5.7,0.099,Marvin Williams,3,1986-06-19,21.536377
4,2009,NaN,0.063,81.

In [21]:
data = data.merge(career[['player_id','season','player_name','birth_date']],on=['player_id','season'],how='left')

In [22]:
adv = pd.read_csv('../Data/nba_advanced_boxscores.csv')

In [23]:
data = data.merge(adv[['boxscore','date']],on='boxscore',how='left')

Extract game dates from the boxscore data into a separate list in datetime

In [24]:
bs = []
game_dates = []
for box in data.boxscore.unique():
    bs.append(box)
    yr = box[:4]
    mn = box[4:6]
    dy = box[6:-4]
    date = '{}/{}/{}'.format(mn,dy,yr)
    date = pd.to_datetime(date)
    game_dates.append(date)

Append game dates into original boxscores dataframe

In [25]:
new_game_dates = pd.DataFrame({'boxscore':bs,
                              'date':game_dates})

In [26]:
data = data.merge(new_game_dates,on=['boxscore']).drop(columns=['date_x']).rename(columns={'date_y':'date'})

Convert player's birth date into datetime 

In [27]:
data.birth_date = pd.to_datetime(data.birth_date)

Take date and birthday of the player in order to calculate player's age for specific game date into a list

In [28]:
ages = []
for game_date, bday in zip(data.date,data.birth_date):
    age = (game_date - bday) / np.timedelta64(1,'Y')
    ages.append(age)

Save player age into the boxscore dataframe

In [29]:
data['age'] = ages

Reorder columns for easier interpretation 

In [30]:
data = data[['player_id','player_name','boxscore','date','season','team_abbr','assist_percentage','assists',
             'block_percentage','blocks','box_plus_minus','defensive_rating','defensive_rebound_percentage',
             'defensive_rebounds','effective_field_goal_percentage','field_goal_attempts','field_goal_percentage',
             'field_goals','free_throw_attempt_rate','free_throw_attempts','free_throw_percentage','free_throws',
             'minutes_played','offensive_rating','offensive_rebound_percentage','offensive_rebounds',
             'personal_fouls','points','steal_percentage','steals','three_point_attempt_rate','three_point_attempts',
             'three_point_percentage','three_pointers','total_rebound_percentage','total_rebounds',
             'true_shooting_percentage','turnover_percentage','turnovers','two_point_attempts',
             'two_point_percentage','two_pointers','usage_percentage','venue','birth_date','age']]

convert percentages into decimal format

In [31]:
data.assist_percentage = data.assist_percentage / 100
data.block_percentage = data.block_percentage / 100
data.defensive_rebound_percentage = data.defensive_rebound_percentage / 100
data.offensive_rebound_percentage = data.offensive_rebound_percentage / 100
data.steal_percentage = data.steal_percentage / 100
data.total_rebound_percentage = data.total_rebound_percentage / 100
data.turnover_percentage = data.turnover_percentage / 100
data.usage_percentage = data.usage_percentage / 100

Order data in ascending order based on season and date

In [32]:
data = data.sort_values(by=['season','date'])

Group data by `player` and `season` in order to describe how many games a player has currently played in a season, has currently played in his career, or has played for the current team he currently plays for

In [33]:
data['player_game_number_season'] = data.groupby(['player_id','season']).cumcount() + 1
data['player_game_number_career'] = data.groupby(['player_id']).cumcount() + 1
data['player_game_number_team'] = data.groupby(['player_id','team_abbr']).cumcount() + 1

For each player, calculate how many days were in between each game he has played. If it is the first game of their career, then assume 150 days has passed between games (estimated time passed between last amateur and first professional game) 

In [34]:
p_ids = []
bx = []
days = []
for player in data.player_id.unique():
    df = data.loc[data.player_id==player].reset_index(drop=True).sort_values(by='date')
    for n in range(len(df)):
        p_ids.append(player)
        bx.append(df.boxscore[n])
        if n == 0:
            days.append(150)
        else:
            try:
                days.append(round((df.date[n] - df.date[n-1])/np.timedelta64(1,'D')))
            except:
                days.append(-1)

Save this new column `days since last game` to a dataframe

In [35]:
days_bw_games = pd.DataFrame({'player_id':p_ids,
                             'boxscore':bx,
                             'days_since_last_game':days})

View the `days_bw_games` dataframe

In [36]:
days_bw_games.head()

,player_id,boxscore,days_since_last_game
0,mikange01,195310300MLH,150
1,mikange01,195310310NYK,1
2,mikange01,195311010BOS,1
3,mikange01,195311040BLB,3
4,mikange01,195311050SYR,1


Merge `days_bw_games` into the boxscores (main) dataframe

In [37]:
data = data.merge(days_bw_games,on=['player_id','boxscore'])

Change `venue` column from a text to a numerical value for easier processing

In [38]:
data.venue.replace('Away',0,inplace=True)
data.venue.replace('Home',1,inplace=True)

Rename data column from `venue` to `home_game` 

In [39]:
data.rename(columns={'venue':'home_game'},inplace=True)

Reorder the data frame 

In [40]:
data = data[['player_id','player_name','boxscore','date','season','team_abbr','assist_percentage','assists',
             'block_percentage','blocks','box_plus_minus','defensive_rating','defensive_rebound_percentage',
             'defensive_rebounds','effective_field_goal_percentage','field_goal_attempts','field_goal_percentage',
             'field_goals','free_throw_attempt_rate','free_throw_attempts','free_throw_percentage','free_throws',
             'minutes_played','offensive_rating','offensive_rebound_percentage','offensive_rebounds',
             'personal_fouls','points','steal_percentage','steals','three_point_attempt_rate','three_point_attempts',
             'three_point_percentage','three_pointers','total_rebound_percentage','total_rebounds',
             'true_shooting_percentage','turnover_percentage','turnovers','two_point_attempts',
             'two_point_percentage','two_pointers','usage_percentage','home_game','birth_date','age',
             'player_game_number_season','player_game_number_career','player_game_number_team','days_since_last_game']]

Connect to the database

In [26]:
mypasswd = getpass.getpass()

# Then connects to the DB
conn = psycopg2.connect(database = 'cs20_group4', 
                              user = 'fhfrf', 
                              host = 'pgsql.dsa.lan',
                              password = mypasswd)

# Then remove the password from computer memory
del mypasswd

········


In [27]:
cursor = conn.cursor()

Create a table called `nba_player_boxscores`

In [28]:
create_table = """
DROP TABLE IF EXISTS nba_player_boxscores;
CREATE TABLE nba_player_boxscores (
player_id varchar(20)
, player_name varchar(100)
, boxscore varchar(100)
, date TIMESTAMP
, season numeric
, team_abbr varchar(10)
, assist_percentage numeric
, assists numeric
, block_percentage numeric
, blocks numeric
, box_plus_minus numeric
, defensive_rating numeric
, defensive_rebound_percentage numeric
, defensive_rebounds numeric
, effective_field_goal_percentage numeric
, field_goal_attempts numeric
, field_goal_percentage numeric
, field_goals numeric
, free_throw_attempt_rate numeric
, free_throw_attempts numeric
, free_throw_percentage numeric
, free_throws numeric
, minutes_played numeric
, offensive_rating numeric
, offensive_rebound_percentage numeric
, offensive_rebounds numeric
, personal_fouls numeric
, points numeric
, steal_percentage numeric
, steals numeric
, three_point_attempt_rate numeric
, three_point_attempts numeric
, three_point_percentage numeric
, three_pointers numeric
, total_rebound_percentage numeric
, total_rebounds numeric
, true_shooting_percentage numeric
, turnover_percentage numeric
, turnovers numeric
, two_point_attempts numeric
, two_point_percentage numeric
, two_pointers numeric
, usage_percentage numeric
, home_game numeric
, birth_date TIMESTAMP
, age numeric
, player_game_number_season numeric
, player_game_number_career numeric
, player_game_number_team numeric
, days_since_last_game numeric
)
"""

In [29]:
cursor.execute(create_table)

In [30]:
conn.commit()

In [8]:
mypasswd = getpass.getpass()

# Then connects to the DB
conn = psycopg2.connect(database = 'cs20_group4', 
                              user = 'fhfrf', 
                              host = 'pgsql.dsa.lan',
                              password = mypasswd)

# Then remove the password from computer memory
del mypasswd

········


In [9]:
cursor = conn.cursor()

Write the dataset that we have performed all data carpentry on into a CSV file so we don't have to run a large chunk of the notebook when we need access to the data.

In [33]:
data.to_csv('testfile.csv', index = False)

In [3]:
master_data = pd.read_csv('testfile.csv')

The three cells below slice the data set into three different subsets that allow us to filter and fill in NaNs with None values on our data. The slicing was required to circumvent issues with memory consumption processing large datasets.

In [4]:
testData = pd.DataFrame()

data1 = master_data.iloc[0:200000]
data1 = data1.where(pd.notnull(data1), None)
testData = testData.append(data1)
del data1



In [6]:
data2 = master_data.iloc[1*10**5:2*10**5]
data2 = data2.where(pd.notnull(data2), None)

testData = testData.append(data2)

In [7]:
data3 = master_data.where(pd.notnull(master_data), None)

data3.shape

(1351639, 50)

Insert data into the `nba_player_boxscores` table

In [10]:
#df = data.where(pd.notnull(data),None)
#data.dropna(subset=['birth_date'], inplace=True)
data3 = data3.loc[data3.birth_date.notnull()]
INSERT_SQL = 'INSERT INTO nba_player_boxscores'
INSERT_SQL += '(player_id'
INSERT_SQL += ', player_name'
INSERT_SQL += ', boxscore'
INSERT_SQL += ', date'
INSERT_SQL += ', season'
INSERT_SQL += ', team_abbr'
INSERT_SQL += ', assist_percentage'
INSERT_SQL += ', assists'
INSERT_SQL += ', block_percentage'
INSERT_SQL += ', blocks'
INSERT_SQL += ', box_plus_minus'
INSERT_SQL += ', defensive_rating'
INSERT_SQL += ', defensive_rebound_percentage'
INSERT_SQL += ', defensive_rebounds'
INSERT_SQL += ', effective_field_goal_percentage'
INSERT_SQL += ', field_goal_attempts'
INSERT_SQL += ', field_goal_percentage'
INSERT_SQL += ', field_goals'
INSERT_SQL += ', free_throw_attempt_rate'
INSERT_SQL += ', free_throw_attempts'
INSERT_SQL += ', free_throw_percentage'
INSERT_SQL += ', free_throws'
INSERT_SQL += ', minutes_played'
INSERT_SQL += ', offensive_rating'
INSERT_SQL += ', offensive_rebound_percentage'
INSERT_SQL += ', offensive_rebounds'
INSERT_SQL += ', personal_fouls'
INSERT_SQL += ', points'
INSERT_SQL += ', steal_percentage'
INSERT_SQL += ', steals'
INSERT_SQL += ', three_point_attempt_rate'
INSERT_SQL += ', three_point_attempts'
INSERT_SQL += ', three_point_percentage'
INSERT_SQL += ', three_pointers'
INSERT_SQL += ', total_rebound_percentage'
INSERT_SQL += ', total_rebounds'
INSERT_SQL += ', true_shooting_percentage'
INSERT_SQL += ', turnover_percentage'
INSERT_SQL += ', turnovers'
INSERT_SQL += ', two_point_attempts'
INSERT_SQL += ', two_point_percentage'
INSERT_SQL += ', two_pointers'
INSERT_SQL += ', usage_percentage'
INSERT_SQL += ', home_game'
INSERT_SQL += ', birth_date'
INSERT_SQL += ', age'
INSERT_SQL += ', player_game_number_season'
INSERT_SQL += ', player_game_number_career'
INSERT_SQL += ', player_game_number_team'
INSERT_SQL += ', days_since_last_game) VALUES'
INSERT_SQL += '(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,'
INSERT_SQL += '%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,'
INSERT_SQL += '%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,'
INSERT_SQL += '%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,'
INSERT_SQL += '%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'

with conn, conn.cursor() as cursor:
    for row in data3.itertuples(index=False, name=None):
        cursor.execute(INSERT_SQL,row)